In [1]:
%load_ext autoreload
%autoreload 2
from infovis21.datamodel.Track import Track
from infovis21.datamodel.User import User
from infovis21.mongodb import MongoAccess as ma
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

ModuleNotFoundError: No module named 'flask'

In [ ]:
# # I know the without is a bit redundant as we could just use the total size of the collection which would be faster but it's also a sanity check for the edge case values of genres

# pipeline_with_genre = [
#     {'$set': {'n_of_genres': {'$size' : {'$ifNull': [ "$genres", [] ]}} } },
#     {'$match': {'n_of_genres': {'$gt': 0}}}
# ]
# res_with_genre = list(ma.coll_artists.aggregate(pipeline_with_genre))
# print(f'With genre info in artists {len(res_with_genre)}')

# pipeline_without_genre = [
#     {'$set': {'n_of_genres': {'$size' : {'$ifNull': [ "$genres", [] ]}} } },
#     {'$match': {'n_of_genres': {'$eq': 0}}}
# ]
# res_without_genre = list(ma.coll_artists.aggregate(pipeline_without_genre))
# print(f'Without genre info in artists {len(res_without_genre)}')

# res_with_genre = list(ma.coll_tracks.aggregate(pipeline_with_genre))
# print(f'With genre info in tracks {len(res_with_genre)}')
# res_without_genre = list(ma.coll_tracks.aggregate(pipeline_without_genre))
# print(f'Without genre info in tracks {len(res_without_genre)}')


df_genres = pd.DataFrame(ma.get_collection(ma.coll_genres))

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import minmax_scale
import os

# from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score

df = df_genres.drop(['genres', 'duration_ms', 'popularity'], axis=1)

In [ ]:
# unsupervised learning to group similar subgenres together; n_clusters = number of genres

from sklearn.cluster import KMeans
Kmean = KMeans(n_clusters=11, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=None, algorithm='auto')
Kmean.fit(df)

In [ ]:
df['label'] = pd.DataFrame(Kmean.labels_)

df = pd.concat([df_genres.genres, df],axis=1)

In [ ]:
# labeling of main genre based on genres 

df.label = df.label.replace(0, "Pop")

df.label = df.label.replace(1, "Classical")
 
df.label = df.label.replace(2, "Jazz")

df.label = df.label.replace(3, "Reggae")

df.label = df.label.replace(4, "Country")

df.label = df.label.replace(5, "Rock")

df.label = df.label.replace(6, "Electronic")

df.label = df.label.replace(7, "Blues")

df.label = df.label.replace(8, "Electronic")

df.label = df.label.replace(9, "Hiphop")

df.label = df.label.replace(10, "Classical")


In [ ]:
# picking color for each genre.

def label_color(df):
    if df['label'] == "Classical" :
        return '#212F3D'
    if df['label'] == "Electronic":
        return '#FF5733'
    if df['label'] == "Jazz" :
        return '#FFE033'
    if df['label'] == "Hiphop":
        return '#2ECC71'
    if df['label'] == "Blues":
        return '#3364FF'
    if df['label'] == "Rock":
        return '#C367D9'
    if df['label'] == "Pop":
        return '#E74C3C'
    if df['label'] == "Country":
        return '#935116'
    if df['label'] == "Reggae":
        return '#FDFEFE'

In [ ]:
# applies new row for color codes for each genre
df['label_color'] = df.apply(label_color, axis=1)